##DATA TRANSFORMATION

In [1]:
from pyspark.sql import SparkSession, DataFrame, Window, functions as F
from pyspark.sql.functions import when, col, lower, regexp_replace
from google.colab import drive

**PEMBUATAN SESI SPARK**

In [4]:
spark = SparkSession.builder.appName('yelp review').getOrCreate()

cells ini berfungsi sebagai titik awal untuk menggunakan semua fitur Spark seperti membaca data dan melakukan transformasi.

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


**LOADING DATASET**

In [ ]:
df_review = spark.read.json('drive/MyDrive/Yelp_review/Yelp-Extracted/yelp_academic_dataset_review.json')

In [ ]:
df_business = spark.read.json('drive/MyDrive/Yelp_review/Yelp-Extracted/yelp_academic_dataset_business.json')

In [ ]:
df_checkin = spark.read.json('drive/MyDrive/Yelp_review/Yelp-Extracted/yelp_academic_dataset_checkin.json')

In [ ]:
df_tip = spark.read.json('drive/MyDrive/Yelp_review/Yelp-Extracted/yelp_academic_dataset_tip.json')

In [ ]:
df_user = spark.read.json('drive/MyDrive/Yelp_review/Yelp-Extracted/yelp_academic_dataset_user.json')

**PENUNJUKAN DATASET**

In [ ]:
df_review.show(5)
df_business.show(5)
df_checkin.show(5)
df_tip.show(5)
df_user.show(5)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [ ]:
df_review.printSchema()
df_business.printSchema()
df_checkin.printSchema()
df_tip.printSchema()
df_user.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

root
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointment

##TRANSFORMATION DATASET

In [ ]:
# Join reviews with businesses
df_join = df_review.join(df_business, on="business_id", how="left")

# Join with users
df_join = df_join.join(df_user, on="user_id", how="left")

# Join with checkins
df_join = df_join.join(df_checkin, on="business_id", how="left")

# Join with tips
df_join = df_join.join(df_tip, on=["business_id", "user_id"], how="left")

df_join.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: 

cells ini menggabungkan beberapa dataset dari review, bisnis, user, check-in, dan tip, menggunakan left join berdasarkan ID yang sama. Hasilnya adalah satu DataFrame lengkap yang berisi semua informasi terkait.

In [ ]:
df_join.show(5)

+--------------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+--------------------+--------------------+------------+--------------------+-------+----------+-----------+--------------------+-----------+------------+-----+-----+-------------+---------------+---------------+----------------+--------------+---------------+---------------+---------------+-----------------+----------------+------------------+-----------------+------+--------------------+----+--------------------+------+-------+------------+------+-------------------+--------------------+----------------+----+----+
|         business_id|             user_id|cool|               date|funny|           review_id|stars|                text|useful|             address|          attributes|          categories|        city|               hours|is_open|  latitude|  longitude|                name|postal_code|review_count|stars|state|average_stars|

In [ ]:
df_join.count()

7103760

In [ ]:
df_business.count()

150346

In [ ]:
df_review.count()

6990280

In [ ]:
df_review = df_review.withColumnRenamed("stars", "review_stars")
df_business = df_business.withColumnRenamed("stars", "business_stars")

joined_df = df_review.join(df_business, on="business_id", how="inner")

joined_df.select(
    "review_id",
    "user_id",
    "review_stars",
    "text",
    "name",
    "city",
    "categories",
    "business_stars",
    "review_count",
    "is_open"
).show(10, truncate=True)

print("Joined row count:", joined_df.count())

+--------------------+--------------------+------------+--------------------+-----------------+---------------+--------------------+--------------+------------+-------+
|           review_id|             user_id|review_stars|                text|             name|           city|          categories|business_stars|review_count|is_open|
+--------------------+--------------------+------------+--------------------+-----------------+---------------+--------------------+--------------+------------+-------+
|hoLKem4XpXGt_X4li...|415SXXbQrl2-VudHa...|         4.0|Ate here for the ...|Frankie's Raw Bar|New Port Richey|Seafood, Restaura...|           4.5|          24|      1|
|gzXaxjjTnrJJuG_gq...|ey8tdgUgC2tFqw2df...|         5.0|Absolutely fantas...|Frankie's Raw Bar|New Port Richey|Seafood, Restaura...|           4.5|          24|      1|
|cpyLQlML0idHzCsCi...|wNo_joX2evLhIYE4h...|         3.0|I was sadly disap...|Frankie's Raw Bar|New Port Richey|Seafood, Restaura...|           4.5|        

code cells ini mengganti nama kolom stars di dataset review dan bisnis agar tidak membingungkan, setelah itu dilakukan inner join berdasarkan business_id

In [ ]:
joined_df.select('categories','name','business_stars','review_stars','text').show(5)

+--------------------+--------------------+--------------+------------+--------------------+
|          categories|                name|business_stars|review_stars|                text|
+--------------------+--------------------+--------------+------------+--------------------+
|Restaurants, Brea...|Turning Point of ...|           3.0|         3.0|If you decide to ...|
|Active Life, Cycl...|Body Cycle Spinni...|           5.0|         5.0|I've taken a lot ...|
|Restaurants, Brea...|   Kettle Restaurant|           3.5|         3.0|Family diner. Had...|
|Halal, Pakistani,...|               Zaika|           4.0|         5.0|Wow!  Yummy, diff...|
|Sandwiches, Beer,...|                Melt|           4.0|         4.0|Cute interior and...|
+--------------------+--------------------+--------------+------------+--------------------+
only showing top 5 rows



In [ ]:
df_restaurants = joined_df.filter(F.col("categories").rlike("(?i).*restaurant.*"))


code cells ini memfilter data hasil join agar hanya menampilkan bisnis yang berhubungan dengan restoran, mengunakan wildcard dengan regular expression yang tidak peka huruf besar-kecil untuk mencocokkan kategori yang mengandung kata “restaurant”, karena terdapat banyak variasi nama kategori.

In [ ]:
df_restaurants.select('categories').distinct().show(5)

+--------------------+
|          categories|
+--------------------+
|Mediterranean, Fo...|
|Seafood, Restaura...|
|Restaurants, Comf...|
|Restaurants, Crep...|
|Specialty Food, F...|
+--------------------+
only showing top 5 rows



In [ ]:
df_restaurants.show(10)

+--------------------+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+-------+----------+-----------+-----------------+-----------+------------+--------------+-----+
|         business_id|cool|               date|funny|           review_id|review_stars|                text|useful|             user_id|         address|          attributes|          categories|           city|               hours|is_open|  latitude|  longitude|             name|postal_code|review_count|business_stars|state|
+--------------------+----+-------------------+-----+--------------------+------------+--------------------+------+--------------------+----------------+--------------------+--------------------+---------------+--------------------+-------+----------+-----------+-----------------+-----------+------------+--------------+-----+
|---kPU91CF4Lq2-

In [ ]:
df_restaurants.select('date','text','review_stars','city','state').show(5)

+-------------------+--------------------+------------+------------+-----+
|               date|                text|review_stars|        city|state|
+-------------------+--------------------+------------+------------+-----+
|2018-07-07 22:09:11|If you decide to ...|         3.0| North Wales|   PA|
|2014-02-05 20:30:30|Family diner. Had...|         3.0|      Tucson|   AZ|
|2015-01-04 00:01:03|Wow!  Yummy, diff...|         5.0|Philadelphia|   PA|
|2017-01-14 20:54:15|Cute interior and...|         4.0| New Orleans|   LA|
|2015-09-23 23:10:31|I am a long term ...|         1.0|Philadelphia|   PA|
+-------------------+--------------------+------------+------------+-----+
only showing top 5 rows



In [ ]:
df_restaurants.select('text').count()

4724684

In [ ]:
df_labeled = df_restaurants.withColumn(
    "sentiment",
    when(col("review_stars") <= 2, 0)  # Negative
    .when(col("review_stars") == 3, 1) # Neutral
    .when(col("review_stars") >= 4, 2) # Positive
).withColumn(
    "sentiment_label",
    when(col("review_stars") <= 2, "Negative")
    .when(col("review_stars") == 3, "Neutral")
    .when(col("review_stars") >= 4, "Positive")
)

# Show sample
df_labeled.select("text", "review_stars", "sentiment", "sentiment_label").show(10, truncate=50)

+--------------------------------------------------+------------+---------+---------------+
|                                              text|review_stars|sentiment|sentiment_label|
+--------------------------------------------------+------------+---------+---------------+
|If you decide to eat here, just be aware it is ...|         3.0|        1|        Neutral|
|Family diner. Had the buffet. Eclectic assortme...|         3.0|        1|        Neutral|
|Wow!  Yummy, different,  delicious.   Our favor...|         5.0|        2|       Positive|
|Cute interior and owner (?) gave us tour of upc...|         4.0|        2|       Positive|
|I am a long term frequent customer of this esta...|         1.0|        0|       Negative|
|Amazingly amazing wings and homemade bleu chees...|         5.0|        2|       Positive|
|Had a party of 6 here for hibachi. Our waitress...|         3.0|        1|        Neutral|
|Locals recommended Milktooth, and it's an amazi...|         4.0|        2|     

code cells ini memberikan label sentimen pada setiap review restoran berdasarkan rating-nya, review dengan 1–2 bintang diberi label Negative, 3 bintang sebagai Neutral, dan 4–5 bintang sebagai Positive, hasilnya ditambahkan sebagai nilai numerik dan label teks, lalu ditampilkan beberapa contohnya.

In [ ]:
df_labeled.select('date','text','city','state','review_stars','sentiment','sentiment_label').show(5)

+-------------------+--------------------+------------+-----+------------+---------+---------------+
|               date|                text|        city|state|review_stars|sentiment|sentiment_label|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
|2018-07-07 22:09:11|If you decide to ...| North Wales|   PA|         3.0|        1|        Neutral|
|2014-02-05 20:30:30|Family diner. Had...|      Tucson|   AZ|         3.0|        1|        Neutral|
|2015-01-04 00:01:03|Wow!  Yummy, diff...|Philadelphia|   PA|         5.0|        2|       Positive|
|2017-01-14 20:54:15|Cute interior and...| New Orleans|   LA|         4.0|        2|       Positive|
|2015-09-23 23:10:31|I am a long term ...|Philadelphia|   PA|         1.0|        0|       Negative|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
only showing top 5 rows



##SAVING DATASET

In [ ]:
df_labeled.select('date','text','city','state','review_stars','sentiment','sentiment_label').coalesce(1).write.parquet("restaurant_data_parquet",mode="overwrite")

code cells ini menyimpan data review restoran yang dipilih seperti tanggal, teks, kota, provinsi, rating, dan sentimen ke dalam satu file Parquet bernama "restaurant_data_parquet".

In [ ]:
df_labeled = spark.read.parquet("/content/drive/MyDrive/restaurant_data_parquet/part-00000-39c7b949-dd62-4da8-9bcf-b26ce00f6ee5-c000.snappy.parquet")

In [ ]:
df_labeled.show(5)

+-------------------+--------------------+------------+-----+------------+---------+---------------+
|               date|                text|        city|state|review_stars|sentiment|sentiment_label|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
|2018-07-07 22:09:11|If you decide to ...| North Wales|   PA|         3.0|        1|        Neutral|
|2014-02-05 20:30:30|Family diner. Had...|      Tucson|   AZ|         3.0|        1|        Neutral|
|2015-01-04 00:01:03|Wow!  Yummy, diff...|Philadelphia|   PA|         5.0|        2|       Positive|
|2017-01-14 20:54:15|Cute interior and...| New Orleans|   LA|         4.0|        2|       Positive|
|2015-09-23 23:10:31|I am a long term ...|Philadelphia|   PA|         1.0|        0|       Negative|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
only showing top 5 rows



In [ ]:
df_state_counts = (
    df_labeled
    .groupBy("state", "sentiment_label")
    .count()
    .groupBy("state")
    .pivot("sentiment_label", ["Negative", "Positive", "Neutral"])
    .sum("count")
    .fillna(0)
)

df_state_counts = df_state_counts.withColumn(
    "total_reviews",
    F.col("Negative") + F.col("Positive") + F.col("Neutral")
)

df_state_counts.orderBy(F.desc("total_reviews")).show(20, truncate=False)

+-----+--------+--------+-------+-------------+
|state|Negative|Positive|Neutral|total_reviews|
+-----+--------+--------+-------+-------------+
|PA   |227705  |732388  |140183 |1100276      |
|FL   |162630  |546874  |82637  |792141       |
|LA   |98700   |396429  |63218  |558347       |
|TN   |89233   |296904  |48572  |434709       |
|MO   |73149   |239932  |41532  |354613       |
|IN   |65217   |232019  |38607  |335843       |
|AZ   |61095   |176073  |29920  |267088       |
|NV   |55555   |162523  |25571  |243649       |
|CA   |37587   |152257  |21904  |211748       |
|NJ   |42096   |109212  |19898  |171206       |
|ID   |22339   |67363   |10380  |100082       |
|AB   |14239   |44223   |10420  |68882        |
|DE   |12235   |30892   |6032   |49159        |
|IL   |9911    |22715   |4244   |36870        |
|NC   |24      |4       |1      |29           |
|HI   |7       |12      |0      |19           |
|CO   |0       |6       |6      |12           |
|MT   |0       |6       |0      |6      

Kode ini mengelompokkan review restoran berdasarkan state dan label sentimen, menghitung jumlahnya, lalu memutar tabel agar setiap sentimen menjadi kolom tersendiri (Negative, Positive, Neutral). Nilai kosong diganti dengan 0

## UNDERSAMPLING DATA

In [5]:
df_pa_balanced = spark.read.parquet('/content/drive/MyDrive/restaurant_data_parquet/part-00000-39c7b949-dd62-4da8-9bcf-b26ce00f6ee5-c000.snappy.parquet')

In [7]:
df_pa_balanced.groupBy("sentiment_label").count().show()

+---------------+-------+
|sentiment_label|  count|
+---------------+-------+
|        Neutral| 543126|
|       Positive|3209836|
|       Negative| 971722|
+---------------+-------+



In [8]:
df_pa_balanced.show(5)

+-------------------+--------------------+------------+-----+------------+---------+---------------+
|               date|                text|        city|state|review_stars|sentiment|sentiment_label|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
|2018-07-07 22:09:11|If you decide to ...| North Wales|   PA|         3.0|        1|        Neutral|
|2014-02-05 20:30:30|Family diner. Had...|      Tucson|   AZ|         3.0|        1|        Neutral|
|2015-01-04 00:01:03|Wow!  Yummy, diff...|Philadelphia|   PA|         5.0|        2|       Positive|
|2017-01-14 20:54:15|Cute interior and...| New Orleans|   LA|         4.0|        2|       Positive|
|2015-09-23 23:10:31|I am a long term ...|Philadelphia|   PA|         1.0|        0|       Negative|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
only showing top 5 rows



In [10]:
df_pa = df_pa_balanced.filter(col("state") == "PA")

In [11]:
def undersample(df: DataFrame, label_col: str, n: int) -> DataFrame:
    sampled_dfs = []
    for label in df.select(label_col).distinct().rdd.flatMap(lambda x: x).collect():
        sampled = df.filter(col(label_col) == label).sample(False, 1.0).limit(n)
        sampled_dfs.append(sampled)
    return sampled_dfs[0].union(sampled_dfs[1]).union(sampled_dfs[2])

df_pa_balanced = undersample(df_pa, "sentiment_label", 20000)

In [12]:
df_pa_balanced.groupBy("sentiment_label").count().show()

+---------------+-----+
|sentiment_label|count|
+---------------+-----+
|        Neutral|20000|
|       Positive|20000|
|       Negative|20000|
+---------------+-----+



In [13]:
df_pa_balanced.show(5)

+-------------------+--------------------+------------+-----+------------+---------+---------------+
|               date|                text|        city|state|review_stars|sentiment|sentiment_label|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
|2018-07-07 22:09:11|If you decide to ...| North Wales|   PA|         3.0|        1|        Neutral|
|2013-10-03 16:24:30|A couple friends ...|Philadelphia|   PA|         3.0|        1|        Neutral|
|2017-08-08 01:37:37|Sometimes this fo...|West Chester|   PA|         3.0|        1|        Neutral|
|2014-07-13 18:28:58|After trying a fe...|Philadelphia|   PA|         3.0|        1|        Neutral|
|2016-01-30 15:10:41|Great food. Terri...|    Broomall|   PA|         3.0|        1|        Neutral|
+-------------------+--------------------+------------+-----+------------+---------+---------------+
only showing top 5 rows



In [ ]:
df_pa_balanced.select('text','sentiment').coalesce(1).write.parquet("restaurant_data_pa",mode="overwrite")

##DATA SIAP TRAINING

In [ ]:
df_svm_unclean = df_pa_balanced.withColumn(
    "clean_text",
    lower(regexp_replace(col("text"), r"[^a-zA-Z\s]", ""))
)

code cells ini membersihkan data teks dengan menghapus semua karakter selain huruf (seperti angka dan tanda baca) serta mengubah seluruh teks menjadi huruf kecil. Hasilnya disimpan pada kolom baru bernama clean_text yang akan digunakan untuk training model SVM

In [ ]:
df_svm_unclean.show(5)

+-------------------+--------------------+------------+-----+------------+---------+---------------+--------------------+
|               date|                text|        city|state|review_stars|sentiment|sentiment_label|          clean_text|
+-------------------+--------------------+------------+-----+------------+---------+---------------+--------------------+
|2018-07-07 22:09:11|If you decide to ...| North Wales|   PA|         3.0|        1|        Neutral|if you decide to ...|
|2014-02-05 20:30:30|Family diner. Had...|      Tucson|   AZ|         3.0|        1|        Neutral|family diner had ...|
|2015-01-04 00:01:03|Wow!  Yummy, diff...|Philadelphia|   PA|         5.0|        2|       Positive|wow  yummy differ...|
|2017-01-14 20:54:15|Cute interior and...| New Orleans|   LA|         4.0|        2|       Positive|cute interior and...|
|2015-09-23 23:10:31|I am a long term ...|Philadelphia|   PA|         1.0|        0|       Negative|i am a long term ...|
+-------------------+---

In [ ]:
df_svm = df_svm_unclean.select('clean_text','sentiment')

In [ ]:
df_svm.show(5)

+--------------------+---------+
|          clean_text|sentiment|
+--------------------+---------+
|if you decide to ...|        1|
|family diner had ...|        1|
|wow  yummy differ...|        2|
|cute interior and...|        2|
|i am a long term ...|        0|
+--------------------+---------+
only showing top 5 rows



In [ ]:
df_svm.coalesce(1).write.parquet("svm_data_pa",mode="overwrite")

In [ ]:
df_bert = df_pa_balanced.select('text','sentiment')

In [ ]:
df_bert.show(5)

+--------------------+---------+
|                text|sentiment|
+--------------------+---------+
|If you decide to ...|        1|
|Family diner. Had...|        1|
|Wow!  Yummy, diff...|        2|
|Cute interior and...|        2|
|I am a long term ...|        0|
+--------------------+---------+
only showing top 5 rows



In [ ]:
df_bert.coalesce(1).write.parquet("bert_data_pa",mode="overwrite")